$\Delta u + k^2(1+q) u = f $ in $\Omega = [0,1]^2$    
$u = 0 $ on $\partial \Omega$

In [1]:
import scipy as sp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres
import time

In [2]:
k = 1   # wavenumber
N = 100 # 格点数
h = 1/N # 间隔

In [3]:
def q_gen_example(N):
    q = np.zeros((N+1,N+1))
    q_value = 0.02
    x1,x2,x3,y1,y2,y3,y4 = 0.2,0.4,0.7,0.2,0.3,0.6,0.7
    q[int(x1*N):int(x2*N),int(y1*N):int(y4*N)] = q_value
    q[int(x2*N):int(x3*N),int(y2*N):int(y3*N)] = q_value
    return q
def q_generation(N,method = 1):
    if method == 1:
        return q_gen_example(N)
    print('method error')
# q = q_generation(N)
# sns.heatmap(q, xticklabels=False, yticklabels=False)

$u = \sin(x\pi)\sin(y\pi)$  
$f = \Delta u + (1+q) u = (1+q-2\pi^2)u$

In [4]:
def u_gen(N):
    u = np.zeros((N+1,N+1))
    for i in range(1,N):
        for j in range(1,N):
            u[i,j] = np.sin(i*np.pi/N)*np.sin(j*np.pi/N)
    return u

# u_truth = u_gen(N)
# sns.heatmap(u_truth, xticklabels=False, yticklabels=False)

In [5]:
def f_gen_1(N,q,u):
    return (1+q-2*np.pi*np.pi)*u

# f = f_gen_1(N,q,u_truth)
# sns.heatmap(f, xticklabels=False, yticklabels=False)

### Method1 五点格式
$(u_{i+1,j} +u_{i-1,j} +u_{i,j+1} +u_{i,j-1} - 4u_{i,j})/h^2 + (1+q_{i,j})u_{i,j} = f_{i,j}$  
$M_1 = \text{Tri}\,(T_1,T_1,T_2,\cdots,T_{N-1};I,I,\cdots,I;I,I,\cdots,I)$  
$T_{i} = \text{Tri}\,(1+q_{i,1}-\frac{4}{h^2},1+q_{i,2}-\frac{4}{h^2},\cdots,1+q_{i,n}-\frac{4}{h^2};\frac{1}{h^2},\frac{1}{h^2},\cdots,\frac{1}{h^2};\frac{1}{h^2},\frac{1}{h^2},\cdots,\frac{1}{h^2})$  
$U = (u_{1,1},u_{1,2},\ldots ,u_{n,n})^T$  
$F_1 = (f_{1,1},f_{1,2},\ldots ,f_{n,n})^T$  
$M_1U = F$

In [6]:
def Matrix_5(N,q):
    M = N-1
    Q = q[1:-1,1:-1].reshape(M*M,1)
    row,col,data = np.array([]),np.array([]),np.array([])
    for i in range(M*M):
        row = np.append(row,i)
        col = np.append(col,i)
        data = np.append(data,1+Q[i] - 4*N*N)
        if (i+1)%M!=0:
            row = np.append(row,i)
            col = np.append(col,i+1)
            data = np.append(data,N*N)
        if (i+M)<M*M:
            row = np.append(row,i)
            col = np.append(col,i+M)
            data = np.append(data,N*N)
        if i%M!=0:
            row = np.append(row,i)
            col = np.append(col,i-1)
            data = np.append(data,N*N)
        if i-M > -1:
            row = np.append(row,i)
            col = np.append(col,i-M)
            data = np.append(data,N*N)
    return csc_matrix((data,(row,col)),shape = (M*M,M*M))

### Method2 九点格式
$A \,u_{i,j} = u_{i+1,j} +u_{i-1,j} +u_{i,j+1} +u_{i,j-1}  $  
$\frac{A-4I}{h^2} u_{i,j} + k^2(1+q_{i,j})u_{i,j} = f_{i,j} $ 
$B \,u_{i,j} = u_{i+1,j+1} +u_{i-1,j-1} +u_{i-1,j+1} +u_{i+1,j-1}$  
$\frac{A-4I}{h^2} u_{i,j} + \frac{B - 2A+4I}{6h^2}u_{i,j} + k^2(1+q_{i,j})u_{i,j} + \frac{k^2}{12}(A-4I)
(1+q_{i,j})u_{i,j}= f_{i,j} + \frac{1}{12}(A-4I)f_{i,j}$  
$\Rightarrow\quad (B+4A-20I) u_{i,j} +h^2 k^2(0.5A+4I)
(1+q_{i,j})u_{i,j}= h^2 (0.5A+4I)f_{i,j}$

In [7]:
def A(v):
    v[0, :] = 0
    v[-1, :] = 0
    v[:, 0] = 0
    v[:, -1] = 0
    return v[1:-1,2:] + v[:-2,1:-1]+ v[2:,1:-1]+v[1:-1,:-2]

def Matrix_9(N,q,k = 1):
    M = N-1
    h = 1/N
    Q = q[1:-1,1:-1].reshape(M*M,1)
    row,col,data = np.array([]),np.array([]),np.array([])
    value_1 = (1+Q)*h*h*k*k*4 - 20 # 主对角线
    value_2 = 4 + 0.5*h*h*k*k*(1+Q) # A 三对角线&主对角元三对角线
    for i in range(M*M):
        row = np.append(row,i)
        col = np.append(col,i)
        data = np.append(data,value_1[i]) # 主对角线
        
        if (i+M)<M*M:
            row = np.append(row,i)
            col = np.append(col,i+M)
            data = np.append(data,value_2[i]) # 副对角线
            if (i+1)%M!=0:
                row = np.append(row,i)
                col = np.append(col,i+M+1)
                data = np.append(data,1)
            if i%M!=0:
                row = np.append(row,i)
                col = np.append(col,i+M-1)
                data = np.append(data,1)
        if i-M > -1:
            row = np.append(row,i)
            col = np.append(col,i-M)
            data = np.append(data,value_2[i])
            if (i+1)%M!=0:
                row = np.append(row,i)
                col = np.append(col,i-M+1)
                data = np.append(data,1)
            if i%M!=0:
                row = np.append(row,i)
                col = np.append(col,i-M-1)
                data = np.append(data,1)
        if (i+1)%M!=0:
            row = np.append(row,i)
            col = np.append(col,i+1)
            data = np.append(data,value_2[i]) 
        if i%M!=0:
            row = np.append(row,i)
            col = np.append(col,i-1)
            data = np.append(data,value_2[i])
    return csc_matrix((data,(row,col)),shape = (M*M,M*M))

In [8]:
def perform(N,tol=1e-05,restart=20):
    
    def Error(a,a_truth,gap = 1e-10):
        a1 = np.where(a<gap,gap,a)
        a_t1 = np.where(a_truth < gap, gap, a_truth)
        return np.abs(a1/a_t1 - 1)


    q = q_generation(N)
    u_truth = u_gen(N)
    f = f_gen_1(N,q,u_truth)
    h = 1/N
    
    
    time50 = time.time() 
    Matrix5 = Matrix_5(N,q)
    Right5 = f[1:-1,1:-1].reshape((-1,1))
    time51 = time.time() - time50
    u_res,exit = gmres(Matrix5,Right5,tol = tol,restart=restart)
    if exit==0:
        res5 = np.zeros((N+1,N+1))
        res5[1:-1,1:-1] = u_res.reshape(N-1,N-1)
        err5 = np.linalg.norm(Error(res5,u_truth),ord = 2)/(N-1)
    else:
        print('五点格式不收敛')
    time52 = time.time() - time50 - time51
    
    
    time90 = time.time() 
    Matrix9 = Matrix_9(N,q)
    Right9 = ((0.5*A(f)+4*f[1:-1,1:-1])*h*h).reshape((-1,1))
    time91 = time.time() - time90
    u_res,exit = gmres(Matrix9,Right9,tol = tol,restart=restart)
    if exit==0:
        res9 = np.zeros((N+1,N+1))
        res9[1:-1,1:-1] = u_res.reshape(N-1,N-1)
        err9 = np.linalg.norm(Error(res9,u_truth),ord = 2)/(N-1)
    else:
        print('九点格式不收敛')
    time92 = time.time() - time90 - time91
    
    print('N = %d' %N)
    print('五点格式平均相对误差为%f,生成矩阵用时%f,求解矩阵用时%f' %(err5,time51,time52))
    print('九点格式平均相对误差为%f,生成矩阵用时%f,求解矩阵用时%f' %(err9,time91,time92))

In [9]:
perform(100)

N = 100
五点格式平均相对误差为0.000085,生成矩阵用时3.528616,求解矩阵用时0.051035
九点格式平均相对误差为0.000005,生成矩阵用时8.738998,求解矩阵用时0.109914


In [10]:
perform(50)

N = 50
五点格式平均相对误差为0.000347,生成矩阵用时0.442131,求解矩阵用时0.006050
九点格式平均相对误差为0.000004,生成矩阵用时0.842511,求解矩阵用时0.005664


In [11]:
perform(150)

N = 150
五点格式平均相对误差为0.000034,生成矩阵用时12.136201,求解矩阵用时0.153602
九点格式平均相对误差为0.000006,生成矩阵用时34.458225,求解矩阵用时0.156002


* 时间主要花在生成左矩阵上，现在逐点定义的左矩阵的值
* 生成矩阵和求解矩阵的用时都随着N的增加而高阶地变化
* 九点格式误差明显低于五点格式的矩阵方法
* 若求解正问题过程中大量涉及q不变的场景，即不需要生成矩阵，可以体现出此方法的优势
* 之后进一步寻找快速生成左矩阵的方法